In [ ]:
#@title 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/computer-vision-um/cv-um-2021/trabajo-final/

# Trabajo Final: Ejercicio 1

Este ejercicio consiste en trabajar sobre la primera parte del curso para extraer un modelo geométrico de una imagen.

Para este ejercicio vamos a trabajar sobre una imagen de una cancha de Fútbol.

Se probeen las funciones `draw_lines` y `draw_lines_polar` que les pueden servir para dibujar las rectas encontradas. 

\**En los ejercicios del trabajo final es posible utilizar funciones de librerias existentes o código sacado de internet. Siempre y cuando **no se usen para resolver explicitamente lo que pide el ejercicio** y al código sacado de interenet le agreguen el link en comentarios de donde fue sacado ese código.*

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from image_utils import draw_lines, draw_lines_polar

In [ ]:
input_image = 'images/football.png'

### Load Input Image ###

In [ ]:
image = cv2.imread(input_image,cv2.IMREAD_GRAYSCALE)/255

In [ ]:
plt.imshow(image,cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
image_bgr = cv2.imread(input_image,cv2.IMREAD_COLOR)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(image_rgb)
plt.axis('off')
plt.show()

In [ ]:
draw_lines(image_rgb,[[1,4],[-1,140]])

In [ ]:
draw_lines_polar(image_rgb,[[20,120],[100,90]],theta_grad=True)